# Neural audio coding for speech enhancement

In [ ]:
# %pip install -e ./AudioDec
# %clearml-init
#imports
from preprocessing.DataEncoder import DataEncoder
from train.LatentTrainer import LatentTrainer

# Generate encoded dataset

In [ ]:
noise_files = './'
speech_files = './'
encoded_mixed_files = '/dtu/blackhole/15/203189/data/train/speech_code'
encoded_speech_files = '/dtu/blackhole/15/203189/data/train/mixed_code'

data_encoder = DataEncoder(
    speech_files=speech_files,
    noise_files=noise_files,
    encoded_speech_files=encoded_speech_files,
    encoded_mixed_files=encoded_mixed_files
)
data_encoder.encode_data()


In [ ]:
model_dir = '/dtu/blackhole/09/203081/saved_models/'
trainer = LatentTrainer(
    model_dir=model_dir,
    data_dir = encoded_speech_files,
    noise_dir = encoded_mixed_files)

trainer.train(num_epochs = 10)


In [ ]:
from AudioDec.models.autoencoder.AudioDec import Generator
from train.LatentNetwork2 import LatentNetwork
generator = Generator()
latent = LatentNetwork()

In [ ]:
import os
import torch
import torchaudio
from AudioDec.models.autoencoder.AudioDec import Generator
tx_steps = 700000
encoder_checkpoint = os.path.join('./AudioDec','exp', 'autoencoder', 'symAD_vctk_48000_hop300', f"checkpoint-{tx_steps}steps.pkl")
generator = Generator()
generator.load_state_dict(torch.load(encoder_checkpoint, map_location='cpu')['model']['generator'])
data, _ = torchaudio.load("x.wav", backend="soundfile")

latent.load_state_dict(torch.load(f"/Users/madiistvan/Dev/DTU/Fall23/02456-Deep-learning/Poster/Project/Neural-audio-coding-for-speech-enhancement/34 (3).pt", map_location=torch.device('cpu')))
generator.eval()
latent.eval()
x = generator.encoder(data.unsqueeze(0))
x = generator.projector(x)
x, _, _ = generator.quantizer(latent(x))
x = generator.decoder(x)

torchaudio.save("pred.wav", x.detach().cpu().squeeze(1), 48000)

